<a href="https://colab.research.google.com/github/zoebatz/CAP6640_Project/blob/main/NLP_FLAN_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentencepiece
!pip install sacrebleu
!pip install bert_score
!pip install evaluate
!pip install nltk
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np
import os
import kagglehub
from bert_score import score
import sacrebleu
from nltk.translate.meteor_score import meteor_score
import nltk
import evaluate
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Download necessary resources for METEOR
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional but recommended
nltk.download('punkt_tab')



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:
# FLAN T5 for translation, summarization and QA

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [33]:
file_name = 'data_eng_spa_1000'
path = "/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/"+file_name+".csv"
data = pd.read_csv(path)
data.head()

,english,spanish
0,Please don't get me wrong.,"Por favor, no me malentienda."
1,My pencil is red.,Mi lápiz es rojo.
2,Tom doesn't want Mary's pity.,Tom no quiere la lástima de Mary.
3,She made it plain that she wanted to go to col...,Ella dejó bien claro que quería ir a la univer...
4,Take a look around you.,Echa una mirada a tu alrededor.


In [6]:
'''# refresh drive
from google.colab import drive
drive.flush_and_unmount()

# mount drive
drive.mount('/content/drive')

SyntaxError: incomplete input (<ipython-input-6-e95900dfe7cc>, line 1)

In [34]:
# examples from full dataset not included in test data
prompt_examples = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/data/data_spa_samples.csv')
prompt_examples.head()

,english,spanish
0,What color is Tom's hair?,¿De qué color es el pelo de Tom?
1,There are people who think pineapples grow und...,Hay personas que piensan que las piñas crecen ...
2,It was not until the child was fed that he sto...,El niño no paró de llorar hasta que se le dio ...
3,She had to withdraw her son from the club.,Ella tuvo que sacar a su hijo del club.
4,Make yourself at home.,Sentite como en casa.


In [35]:
len(data)

1000

In [36]:
# small sample for testing
test_size = len(data)
data = data[:test_size]

predictions = [] # predicted translation

pred_zero = []
pred_one = []
pred_three = []

In [37]:
import torch
# use gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)




T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [38]:
batch_size = 16

function_name = 'translate'
total = 0




for i in range(0, test_size, batch_size):
    batch_inputs = data['english'].iloc[i:i+batch_size].tolist()


    # zero shot prompts
    zero_shot = [f"""Translate the following English sentence to Spanish:\n{sent}""" for sent in batch_inputs]

    # one shot prompt
    one_shot = [f"""Translate the following English sentences to Spanish.

    English: {prompt_examples['english'][0]}
    Spanish: {prompt_examples['spanish'][0]}

    English: {sent}
    Spanish:""" for sent in batch_inputs]

    # three shot prompt
    three_shot =  [f"""Translate the following English sentences to Spanish.

    English: {prompt_examples['english'][1]}
    Spanish: {prompt_examples['spanish'][1]}

    English: {prompt_examples['english'][2]}
    Spanish: {prompt_examples['spanish'][2]}

    English: {prompt_examples['english'][3]}
    Spanish: {prompt_examples['spanish'][3]}

    English: {sent}
    Spanish:""" for sent in batch_inputs]

    # tokenize batch
    inputs_zero = tokenizer(zero_shot, return_tensors="pt", truncation=True, padding=True).to(device)
    inputs_one = tokenizer(one_shot, return_tensors="pt", truncation=True, padding=True).to(device)
    inputs_three = tokenizer(three_shot, return_tensors="pt", truncation=True, padding=True).to(device)

    # translate for each prompt
    outputs_zero = model.generate(
        input_ids=inputs_zero.input_ids,
        attention_mask=inputs_zero.attention_mask,
        max_length=100,
        num_beams=4,
        early_stopping=True
    )

    outputs_one = model.generate(
        input_ids=inputs_one.input_ids,
        attention_mask=inputs_one.attention_mask,
        max_length=100,
        num_beams=4,
        early_stopping=True
    )

    outputs_three = model.generate(
        input_ids=inputs_three.input_ids,
        attention_mask=inputs_three.attention_mask,
        max_length=100,
        num_beams=4,
        early_stopping=True
    )

    total += batch_size
    print(total)

    # decode outputs
    decoded_zero = tokenizer.batch_decode(outputs_zero, skip_special_tokens=True)
    decoded_one = tokenizer.batch_decode(outputs_one, skip_special_tokens=True)
    decoded_three = tokenizer.batch_decode(outputs_three, skip_special_tokens=True)

    pred_zero.extend(decoded_zero)
    pred_one.extend(decoded_one)
    pred_three.extend(decoded_three)





16
32
48
64
80
96
112
128
144
160
176
192
208
224
240
256
272
288
304
320
336
352
368
384
400
416
432
448
464
480
496
512
528
544
560
576
592
608
624
640
656
672
688
704
720
736
752
768
784
800
816
832
848
864
880
896
912
928
944
960
976
992
1008


In [39]:
print(f'{pred_zero[0]} \n{pred_one[0]} \n{pred_three[0]}')

Por favor, no me olvidas. 
Por favor, no me olvidas. 
Por favor, no me olvidas.


In [15]:
data['spanish'][0]

'Por favor, no me malentienda.'

In [16]:
data['english'][0]

"Please don't get me wrong."

In [43]:
references = data['spanish'[:test_size]].tolist()


In [44]:
# change model name \ to _
model_name = model_name.replace("/", "_")
model_name

'google_flan-t5-base'

In [45]:
# view metrics and save
def view_and_save_metrics(prediction, reference, shot_name):
  # BLEU
  bleu = sacrebleu.corpus_bleu(prediction, [reference])
#  print(f"BLEU: {bleu.score}")

  # Tokenize sentences for METEOR
  prediction_tokens = [nltk.word_tokenize(pred) for pred in prediction]
  reference_tokens = [[nltk.word_tokenize(ref)] for ref in reference]  # Note the double list for reference

  # METEOR (sentence-level average)
  meteor_scores = [
      meteor_score(ref_tokens, pred_tokens)
      for ref_tokens, pred_tokens in zip(reference_tokens, prediction_tokens)
  ]
  avg_meteor = sum(meteor_scores) / len(meteor_scores)

  # ROUGE
  rouge = evaluate.load('rouge')
  rouge_scores = rouge.compute(predictions=prediction, references=reference)
#  print(f"ROUGE: {rouge_scores}")

  # BERT Score
  P_, R_, F1 = score(prediction, reference, lang="el")
  # print(f"Avg BERTScore F1: {F1.mean().item():.4f}")
  bert_score = F1.mean().item()

  metrics = {
      "bleu": bleu.score,
      "meteor": avg_meteor,
      "bert_score": bert_score,
      "model_name": model_name,
      "dataset": file_name,
      "shot": shot_name,
      "size": test_size
  }

  metrics.update(rouge_scores)
  '''
  # Print all metrics
  print("\n--- Evaluation Metrics ---")
  print(f"BLEU:   {bleu.score:.4f}")
  print(f"METEOR: {avg_meteor:.4f}")
  for k, v in rouge_scores.items():
      print(f"{k.upper()}: {v:.4f}")
  '''

  # save metrics to csv file
  metrics_df = pd.DataFrame([metrics])
  metrics_df.to_csv('/content/drive/MyDrive/Colab Notebooks/CAP6640_NLP/metrics/translation/metrics_'+model_name+'_'+file_name+'_'+shot_name+'.csv', index=False)
  return metrics_df

In [46]:
zero_shot_metrics = view_and_save_metrics(pred_zero, references, "zero_shot")
one_shot_metrics = view_and_save_metrics(pred_one, references, "one_shot")
three_shot_metrics = view_and_save_metrics(pred_three, references, "three_shot")


In [47]:
zero_shot_metrics

,bleu,meteor,bert_score,model_name,dataset,shot,size,rouge1,rouge2,rougeL,rougeLsum
0,23.005161,0.471898,0.852853,google_flan-t5-base,data_eng_spa_1000,zero_shot,1000,0.486782,0.269818,0.476331,0.476778


In [48]:
three_shot_metrics

,bleu,meteor,bert_score,model_name,dataset,shot,size,rouge1,rouge2,rougeL,rougeLsum
0,22.453227,0.467986,0.849101,google_flan-t5-base,data_eng_spa_1000,three_shot,1000,0.479292,0.269387,0.469575,0.469908


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load FLAN-T5 base model
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Example sentence to translate
target_sentence = "The children are playing in the garden."

# Define prompts
zero_shot_prompt = f"""Translate this sentence to Spanish:
"{target_sentence}"
"""

one_shot_prompt = f"""Translate the following English sentences to Spanish.

English: I love eating pizza.
Spanish: Me encanta comer pizza.

English: {target_sentence}
Spanish:"""

three_shot_prompt = f"""Translate the following English sentences to Spanish.

English: I love eating pizza.
Spanish: Me encanta comer pizza.

English: He is reading a book in the library.
Spanish: Él está leyendo un libro en la biblioteca.

English: The sun is shining today.
Spanish: El sol brilla hoy.

English: {target_sentence}
Spanish:"""

# Function to translate
def translate(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = model.generate(
        **inputs,
        max_length=100,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Run translations
print("🔹 Zero-Shot Output:")
print(translate(zero_shot_prompt), "\n")

print("🔸 One-Shot Output:")
print(translate(one_shot_prompt), "\n")

print("⭐ Few-Shot (3-Shot) Output:")
print(translate(three_shot_prompt))


🔹 Zero-Shot Output:
Los nios son jugando en el jardn. 

🔸 One-Shot Output:
Los nios están jugando en el jardn. 

⭐ Few-Shot (3-Shot) Output:
Los nios son jugando en el jardn.
